In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/DS_Pipe/bada_dataset.csv'

In [ ]:
#bada dataset is the main original unaltered dataset
data = pd.read_csv(file_path)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (8,10,11,12,13,14,15,16,17,18,35,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#below columns are not useful for various reasons and so dropping them
pruned_data = data.drop(columns = ['OBJECTID', 'FPA_ID', 'SOURCE_SYSTEM',
       'NWCG_REPORTING_UNIT_ID',
       'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
       'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID',
       'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME',
       'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME',
       'COMPLEX_NAME', 'STAT_CAUSE_CODE','OWNER_CODE', 'FIPS_NAME','FIPS_CODE', 'Shape'])

In [ ]:
#columns = ['SOURCE_SYSTEM_TYPE', 'NWCG_REPORTING_AGENCY', 'STAT_CAUSE_DESCR','FIRE_SIZE_CLASS','OWNER_DESCR', 'STATE']

In [ ]:
#temp data is used in kmeans clustering algorithm for discovery time imputation below
temp_data = pruned_data[['STAT_CAUSE_DESCR','FIRE_SIZE_CLASS','OWNER_DESCR','STATE','SOURCE_SYSTEM_TYPE']]
#below columns are found to have significant influence on discovery_time, based on anova test. So, these columns are used for...
#data imputation
columns = ['STAT_CAUSE_DESCR','FIRE_SIZE_CLASS','OWNER_DESCR','STATE','SOURCE_SYSTEM_TYPE']
for column in columns:
  one_hot = pd.get_dummies(temp_data[column],prefix=column)
  temp_data = temp_data.drop(column,axis = 1)
  temp_data = temp_data.join(one_hot)

In [ ]:
#link for reference to the kmeans_missing semi supervised data imputation algorithm
#https://stackoverflow.com/questions/35611465/python-scikit-learn-clustering-with-missing-data
import numpy as np
from sklearn.cluster import KMeans
def kmeans_missing(X, n_clusters, max_iter=10):
    missing = ~np.isfinite(X)
    mu = np.nanmean(X, 0, keepdims=1)
    X_hat = np.where(missing, mu, X)

    for i in range(max_iter):
        if i > 0:
            cls = KMeans(n_clusters, init=prev_centroids)
        else:
            cls = KMeans(n_clusters)
        labels = cls.fit_predict(X_hat)
        centroids = cls.cluster_centers_
        X_hat[missing] = centroids[labels][missing]
        if i > 0 and np.all(labels == prev_labels):
            break
        prev_labels = labels
        prev_centroids = cls.cluster_centers_

    return labels, centroids, X_hat

In [ ]:
#imputing discovery time using the above kmeans clustering algorithm
temp_data['DISCOVERY_TIME'] = pruned_data['DISCOVERY_TIME']
columns = temp_data.columns
temp_data = temp_data.to_numpy()
_,_,temp_data = kmeans_missing(temp_data, 100, max_iter=10)

/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_kmeans.py:1146: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)
/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_kmeans.py:1146: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)
/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_kmeans.py:1146: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)
/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_kmeans.py:1146: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)
/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_kmeans.py:1146: RuntimeWarning: Explicit initial center position passed: performing only one init in KMe

In [ ]:
temp_data = pd.DataFrame(temp_data, columns=columns)

In [ ]:
temp_data.head()

,STAT_CAUSE_DESCR_Arson,STAT_CAUSE_DESCR_Campfire,STAT_CAUSE_DESCR_Children,STAT_CAUSE_DESCR_Debris Burning,STAT_CAUSE_DESCR_Equipment Use,STAT_CAUSE_DESCR_Fireworks,STAT_CAUSE_DESCR_Lightning,STAT_CAUSE_DESCR_Miscellaneous,STAT_CAUSE_DESCR_Missing/Undefined,STAT_CAUSE_DESCR_Powerline,...,STATE_VA,STATE_VT,STATE_WA,STATE_WI,STATE_WV,STATE_WY,SOURCE_SYSTEM_TYPE_FED,SOURCE_SYSTEM_TYPE_INTERAGCY,SOURCE_SYSTEM_TYPE_NONFED,DISCOVERY_TIME
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1300.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,845.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1921.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1600.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1600.0


In [ ]:
pruned_data.columns

Index(['FOD_ID', 'SOURCE_SYSTEM_TYPE', 'NWCG_REPORTING_AGENCY', 'FIRE_YEAR',
       'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME', 'STAT_CAUSE_DESCR',
       'CONT_DATE', 'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS',
       'LATITUDE', 'LONGITUDE', 'OWNER_DESCR', 'STATE', 'COUNTY'],
      dtype='object')

In [ ]:
merged_data = pruned_data.copy()
#commenting below line to skip kmeans imputation step as it would take a lot of time and we are just testing some other feature engineering
#merged_data['DISCOVERY_TIME'] = temp_data['DISCOVERY_TIME']

In [ ]:
merged_data.columns

Index(['FOD_ID', 'SOURCE_SYSTEM_TYPE', 'NWCG_REPORTING_AGENCY', 'FIRE_YEAR',
       'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME', 'STAT_CAUSE_DESCR',
       'CONT_DATE', 'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS',
       'LATITUDE', 'LONGITUDE', 'OWNER_DESCR', 'STATE', 'COUNTY'],
      dtype='object')

In [ ]:
merged_data.to_csv('/content/drive/MyDrive/Data_Science_Pipeline/categorical_data.csv')

In [ ]:
#one_hot_encoded_df =  merged_data.copy()
one_hot_encoded_df = pd.read_csv('/content/drive/MyDrive/DS_Pipe/categorical_data.csv')
#FOD_ID is just a unique identifier and is not useful for prediction
#County information can be caught from latitute and longitude
#CONT_* have missing data and are dropped for now
#Fire size might not be needed as we have fire_size class
one_hot_encoded_df = one_hot_encoded_df.drop(columns = ['FOD_ID','COUNTY','CONT_DATE','CONT_DOY','CONT_TIME','FIRE_SIZE'])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
one_hot_encoded_df.isna().sum()

Unnamed: 0               0
SOURCE_SYSTEM_TYPE       0
NWCG_REPORTING_AGENCY    0
FIRE_YEAR                0
DISCOVERY_DATE           0
DISCOVERY_DOY            0
DISCOVERY_TIME           0
STAT_CAUSE_DESCR         0
FIRE_SIZE_CLASS          0
LATITUDE                 0
LONGITUDE                0
OWNER_DESCR              0
STATE                    0
dtype: int64

In [ ]:
#trying out some feature engineering
#will create new columns based on old columns. So that we can use any type of feature at later point of time

#Time binning
#one_hot_encoded_df['DISCOVERY_TIME_BIN'] = 
fe_dataset = one_hot_encoded_df.copy()
fe_dataset['DISCOVERY_TIME_BIN'] = pd.cut(fe_dataset['DISCOVERY_TIME'],6,labels=['LN','EM','M','A','E','N'])
#discovery doy into seasons binning - 4 seasons
fe_dataset['DISCOVERY_DOY_BIN'] = pd.cut(fe_dataset['DISCOVERY_DOY'],4,labels=['WINTER','SPRING','SUMMER','FALL'])

In [ ]:
def getAngle(lat,longi):
  # print(coord)
  # return
  # lat = coord[9]
  # longi = coord[10]
  mid_point_x, mid_point_y = 37.09, 95.71
  right_point_x, right_point_y = 37.09, 66.57
  ang = math.degrees(math.atan2(longi-mid_point_y, lat-mid_point_x) - math.atan2(right_point_y-mid_point_y, right_point_x-mid_point_x))
  return ang + 360 if ang < 0 else ang

In [ ]:
getAngle([40.03694444,-121.00583333])

[40.03694444, -121.00583333]


In [ ]:
#test.apply(getAngle,axis=1)
fe_dataset.apply(lambda x: getAngle(x.LATITUDE, x.LONGITUDE), axis=1)

0            0.779071
1            0.488615
2            0.501396
3            0.390383
4            0.390348
              ...    
1880460      0.890928
1880461      0.139536
1880462      0.139536
1880463      0.154008
1880464    359.238014
Length: 1880465, dtype: float64

In [ ]:
#creating a single distance metric from latitude and longitude 
#avg_lat = test['LATITUDE'].mean()
#avg_longitude = test['LONGITUDE'].mean()
fe_dataset['lat_lon_dist'] = ((fe_dataset['LATITUDE']-37.09)**2 + (fe_dataset['LONGITUDE']-95.71)**2)**0.5
#creating angle with reference line using latitude and longitude
fe_dataset['lat_lon_angle'] = fe_dataset.apply(lambda x: getAngle(x.LATITUDE, x.LONGITUDE), axis=1)

In [ ]:
fe_dataset.head()

,Unnamed: 0,SOURCE_SYSTEM_TYPE,NWCG_REPORTING_AGENCY,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_DESCR,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE,DISCOVERY_TIME_BIN,DISCOVERY_DOY_BIN,lat_lon_dist,lat_lon_angle
0,0,FED,FS,2005,2453403.5,33,1300.0,Miscellaneous,A,40.036944,-121.005833,USFS,CA,A,WINTER,216.735869,0.779071
1,1,FED,FS,2004,2453137.5,133,845.0,Lightning,A,38.933056,-120.404444,USFS,CA,M,SPRING,216.122303,0.488615
2,2,FED,FS,2004,2453156.5,152,1921.0,Debris Burning,A,38.984167,-120.735556,STATE OR PRIVATE,CA,E,SPRING,216.453844,0.501396
3,3,FED,FS,2004,2453184.5,180,1600.0,Lightning,A,38.559167,-119.913333,USFS,CA,E,SPRING,215.628338,0.390383
4,4,FED,FS,2004,2453184.5,180,1600.0,Lightning,A,38.559167,-119.933056,USFS,CA,E,SPRING,215.648060,0.390348


In [ ]:
#bucketing states into regions
state_region_map = {'CA' : 'Pacific_West', 
'NM' : 'South', 
'OR' : 'North_West', 
'NC' : 'South_East', 
'WY' : 'North_West', 
'CO' : 'Mountain_West', 
'WA' : 'North_West', 
'MT' : 'North_West', 
'UT' : 'Pacific_West', 
'AZ' : 'Pacific_West', 
'SD' : 'Mountain_West',
'AR' : 'South', 
'NV' : 'Pacific_West', 
'ID' : 'North_West', 
'MN' : 'Mountain_West', 
'TX' : 'South', 
'FL' : 'South_East', 
'SC' : 'South_East', 
'LA' : 'South', 
'OK' : 'South', 
'KS' : 'Mountain_West', 
'MO' : 'Mountain_West',
'NE' : 'Mountain_West', 
'MI' : 'Mid_West',
'KY' : 'Mid_West', 
'OH' : 'Mid_West',
'IN' : 'Mid_West', 
'VA' : 'East',
'IL' : 'Mid_West',
'TN' : 'South_East', 
'GA' : 'South_East', 
'AK' : 'North_West', 
'ND' : 'Mountain_West',
'WV' : 'East', 
'WI' : 'Mid_West', 
'AL' : 'South_East', 
'NH' : 'North_East', 
'PA' : 'East', 
'MS' : 'South', 
'ME' : 'North_East', 
'VT' : 'North_East', 
'NY' : 'North_East', 
'IA' : 'Mountain_West', 
'DC' : 'East',
'MD' : 'East', 
'CT' : 'North_East', 
'MA' : 'North_East', 
'NJ' : 'East', 
'HI' : 'North_West', 
'DE' : 'East', 
'PR' : 'South_East', 
'RI' : 'North_East'}
fe_dataset['REGION'] = fe_dataset['STATE'].apply(lambda x:state_region_map[x])

In [ ]:
fe_dataset.head()

,Unnamed: 0,SOURCE_SYSTEM_TYPE,NWCG_REPORTING_AGENCY,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_DESCR,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_DESCR,STATE,DISCOVERY_TIME_BIN,DISCOVERY_DOY_BIN,lat_lon_dist,lat_lon_angle,REGION
0,0,FED,FS,2005,2453403.5,33,1300.0,Miscellaneous,A,40.036944,-121.005833,USFS,CA,A,WINTER,216.735869,0.779071,Pacific_West
1,1,FED,FS,2004,2453137.5,133,845.0,Lightning,A,38.933056,-120.404444,USFS,CA,M,SPRING,216.122303,0.488615,Pacific_West
2,2,FED,FS,2004,2453156.5,152,1921.0,Debris Burning,A,38.984167,-120.735556,STATE OR PRIVATE,CA,E,SPRING,216.453844,0.501396,Pacific_West
3,3,FED,FS,2004,2453184.5,180,1600.0,Lightning,A,38.559167,-119.913333,USFS,CA,E,SPRING,215.628338,0.390383,Pacific_West
4,4,FED,FS,2004,2453184.5,180,1600.0,Lightning,A,38.559167,-119.933056,USFS,CA,E,SPRING,215.648060,0.390348,Pacific_West


In [ ]:
fe_dataset.to_csv('/content/drive/MyDrive/DS_Pipe/feature_engineered_dataset.csv')

In [ ]:
fe_dataset = pd.read_csv('/content/drive/MyDrive/DS_Pipe/feature_engineered_dataset.csv')

In [ ]:
fe_dataset.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'SOURCE_SYSTEM_TYPE',
       'NWCG_REPORTING_AGENCY', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY',
       'DISCOVERY_TIME', 'STAT_CAUSE_DESCR', 'FIRE_SIZE_CLASS', 'LATITUDE',
       'LONGITUDE', 'OWNER_DESCR', 'STATE', 'DISCOVERY_TIME_BIN',
       'DISCOVERY_DOY_BIN', 'lat_lon_dist', 'lat_lon_angle', 'REGION'],
      dtype='object')

In [ ]:
fe_dataset = fe_dataset[fe_dataset.columns[2:]]

In [ ]:
for column in fe_dataset.columns:
  print(column,len(fe_dataset[column].unique()))

SOURCE_SYSTEM_TYPE 3
NWCG_REPORTING_AGENCY 11
FIRE_YEAR 24
DISCOVERY_DATE 8766
DISCOVERY_DOY 366
DISCOVERY_TIME 1442
STAT_CAUSE_DESCR 13
FIRE_SIZE_CLASS 7
LATITUDE 894061
LONGITUDE 997536
OWNER_DESCR 16
STATE 52
DISCOVERY_TIME_BIN 6
DISCOVERY_DOY_BIN 4
lat_lon_dist 1569621
lat_lon_angle 1569558
REGION 8


In [ ]:
#one hot encode remaining
one_hot_encoded_df = fe_dataset.copy()
columns_one_hot_encode =  ['SOURCE_SYSTEM_TYPE','NWCG_REPORTING_AGENCY','STAT_CAUSE_DESCR','FIRE_SIZE_CLASS','OWNER_DESCR','STATE','DISCOVERY_TIME_BIN','DISCOVERY_DOY_BIN','REGION']
for column in columns_one_hot_encode:
  one_hot = pd.get_dummies(one_hot_encoded_df[column],prefix=column)
  # Drop column B as it is now encoded
  one_hot_encoded_df = one_hot_encoded_df.drop(column,axis = 1)
  # Join the encoded df
  one_hot_encoded_df = one_hot_encoded_df.join(one_hot)
one_hot_encoded_df.head()

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,LATITUDE,LONGITUDE,lat_lon_dist,lat_lon_angle,SOURCE_SYSTEM_TYPE_FED,SOURCE_SYSTEM_TYPE_INTERAGCY,...,DISCOVERY_DOY_BIN_SUMMER,DISCOVERY_DOY_BIN_WINTER,REGION_East,REGION_Mid_West,REGION_Mountain_West,REGION_North_East,REGION_North_West,REGION_Pacific_West,REGION_South,REGION_South_East
0,2005,2453403.5,33,1300.0,40.036944,-121.005833,216.735869,0.779071,1,0,...,0,1,0,0,0,0,0,1,0,0
1,2004,2453137.5,133,845.0,38.933056,-120.404444,216.122303,0.488615,1,0,...,0,0,0,0,0,0,0,1,0,0
2,2004,2453156.5,152,1921.0,38.984167,-120.735556,216.453844,0.501396,1,0,...,0,0,0,0,0,0,0,1,0,0
3,2004,2453184.5,180,1600.0,38.559167,-119.913333,215.628338,0.390383,1,0,...,0,0,0,0,0,0,0,1,0,0
4,2004,2453184.5,180,1600.0,38.559167,-119.933056,215.648060,0.390348,1,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
columns_to_scale =  ['FIRE_YEAR','DISCOVERY_DATE','DISCOVERY_DOY','DISCOVERY_TIME','LATITUDE','LONGITUDE','lat_lon_dist','lat_lon_angle']
scaled_df = one_hot_encoded_df.copy()
for column in columns_to_scale:
  scaled_df[column] = (scaled_df[column]-scaled_df[column].mean()) / (scaled_df[column].std())
scaled_df.head()

,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,LATITUDE,LONGITUDE,lat_lon_dist,lat_lon_angle,SOURCE_SYSTEM_TYPE_FED,SOURCE_SYSTEM_TYPE_INTERAGCY,...,DISCOVERY_DOY_BIN_SUMMER,DISCOVERY_DOY_BIN_WINTER,REGION_East,REGION_Mid_West,REGION_Mountain_West,REGION_North_East,REGION_North_West,REGION_Pacific_West,REGION_South,REGION_South_East
0,0.193607,0.139590,-1.462914,-0.517416,0.530333,-1.513488,1.507126,-1.222123,1,0,...,0,1,0,0,0,0,0,1,0,0
1,0.043527,0.030331,-0.352283,-2.056039,0.350518,-1.477513,1.470465,-1.223782,1,0,...,0,0,0,0,0,0,0,1,0,0
2,0.043527,0.038135,-0.141263,1.582551,0.358844,-1.497320,1.490275,-1.223709,1,0,...,0,0,0,0,0,0,0,1,0,0
3,0.043527,0.049636,0.169714,0.497061,0.289615,-1.448135,1.440951,-1.224343,1,0,...,0,0,0,0,0,0,0,1,0,0
4,0.043527,0.049636,0.169714,0.497061,0.289615,-1.449315,1.442129,-1.224343,1,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
scaled_df.to_csv('/content/drive/MyDrive/DS_Pipe/fe_scaled_one_hot_full_2.csv')